# Упражнение 02 : Присоединяйтесь

In [123]:
import pandas as pd
import sqlite3

## создайте подключение к базе данных с помощью библиотеки sqlite3

In [124]:
con = sqlite3.connect('../data/checking-logs.sqlite')

## создайте новую таблицу в базе данных, объединив таблицы «Просмотр страниц» и «Проверка» с помощью всего одного запроса
* в таблице должны быть следующие столбцы: uid, labname, first_commit_ts, first_view_ts.
* first_commit_ts — это просто новое название столбца timestamp в таблице проверки. Он показывает первую фиксацию для конкретной лаборатории и конкретного пользователя
* first_view_ts — это первый визит пользователя в таблицу просмотров, отметка времени, когда пользователь посетил ленту новостей
* status = ’готов’ по-прежнему должен быть фильтром
* numTrials = 1 по-прежнему должно быть фильтром
* Названия лабораторий должны быть из списка: «laba04», «laba04s», «laba05», «laba06», «laba06s», «project1»
* таблица должна содержать только пользователей (идентификаторы с user_*), а не администраторов
* first_commit_ts и first_view_ts должны быть проанализированы как datetime64[ns]

In [125]:
sql = '''
SELECT
    checker.uid, checker.labname,
    checker.timestamp as first_commit_ts,
    pageviews.datetime as first_view_ts
FROM checker
LEFT JOIN pageviews
ON checker.uid = pageviews.uid
WHERE checker.uid LIKE 'user_%'
    AND status = 'ready'
    AND numTrials = 1
    AND labname IN
    ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND (pageviews.datetime = (SELECT MIN(datetime) FROM pageviews
                            WHERE pageviews.uid = checker.uid)
        OR pageviews.datetime IS NULL)
'''
df = pd.read_sql(sql, con, parse_dates=['first_commit_ts', 'first_view_ts'])
df

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


## используя методы Pandas, создайте два фрейма данных: тестовый и контрольный
* у теста должны быть пользователи, у которых есть значения в first_view_ts
* у элемента управления должны быть пользователи, у которых в first_view_ts отсутствуют значения
* замените отсутствующие значения в контрольной группе средним значением first_view_ts тестовых пользователей, мы будем использовать это значение для дальнейшего анализа
* сохраните обе таблицы в базе данных, вы будете использовать их в следующих упражнениях

In [126]:
test = df[df['first_view_ts'].notnull()]
control = df[df['first_view_ts'].isnull()]
control = control.fillna(test['first_view_ts'].mean())

In [127]:
test.to_sql('test', con)
control.to_sql('control', con)

81

## закройте соединение

In [128]:
con.close()